# RAG pipeline with Chroma, Sentence-Transformers, and FLAN-T5 (Colab-ready)

This Colab notebook implements:
- OCR-aware PDF extraction (handles flattened PDFs) using `pdfplumber` + `pytesseract`.
- Text extraction from DOCX and images.
- Chunking and per-document JSON export.
- Embeddings with `sentence-transformers` (recommended: `all-mpnet-base-v2`).
- Vector storage in Chroma (local persistent folder).
- Retrieval (query → embedding → top-K).
- RAG answer synthesis using `google/flan-t5-base`.
- Optional exact-span extraction using a QA model.

**How to use**
1. Open this notebook in Google Colab.
2. (Optional) Change runtime to GPU for faster generation: Runtime → Change runtime type → GPU.
3. Run cells sequentially. Upload files when prompted (or mount Google Drive and modify paths).
4. Use the `rag_answer` function to ask questions.



In [1]:
# Install required packages (Colab)
!pip install -q sentence-transformers chromadb pdfplumber python-docx pytesseract pillow transformers accelerate sentencepiece
# install tesseract binary (Colab / Debian) and poppler-utils for PDF rasterization
!apt-get update -qq && apt-get install -y -qq tesseract-ocr libtesseract-dev poppler-utils


ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\lenovo\\AppData\\Local\\Temp\\pip-unpack-jgi0u2n_\\pillow-12.0.0-cp313-cp313-win_amd64.whl'
Check the permissions.


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
'apt-get' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
# Imports, paths, and settings
import os, json, uuid
from pathlib import Path
from tqdm import tqdm

# Text extraction libs
import pdfplumber, docx
from PIL import Image, ImageFilter, ImageOps
import pytesseract

# Embeddings
from sentence_transformers import SentenceTransformer

# Chroma
import chromadb
from chromadb.config import Settings

# Generation & QA
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import torch

# Folders
DB_DIR = "/content/chroma_db"
JSON_OUTPUT_DIR = "/content/doc_jsons"
os.makedirs(DB_DIR, exist_ok=True)
os.makedirs(JSON_OUTPUT_DIR, exist_ok=True)

# Embedding model (choose accuracy vs speed)
EMBEDDING_MODEL_NAME = "all-mpnet-base-v2"   # recommended for accuracy
# EMBEDDING_MODEL_NAME = "all-MiniLM-L6-v2"  # alternative (faster)

# Generation model (RAG fusion / answer synthesis)
GEN_MODEL_NAME = "google/flan-t5-base"  # use GPU for larger models

# QA span-extraction model (optional)
QA_MODEL_NAME = "deepset/roberta-base-squad2"

# Tesseract settings
pytesseract.pytesseract.tesseract_cmd = "/usr/bin/tesseract"  # colab default
OCR_LANG = "eng"
TESSERACT_COMMON_CONFIG = f"--oem 1 --psm 6 -l {OCR_LANG}"

print("Embedding model:", EMBEDDING_MODEL_NAME)
print("Generation model:", GEN_MODEL_NAME)
print("DB_DIR:", DB_DIR)


Embedding model: all-mpnet-base-v2
Generation model: google/flan-t5-base
DB_DIR: /content/chroma_db


In [3]:
# OCR-aware PDF extractor and helpers

def preprocess_pil_image_for_ocr(pil_img):
    img = pil_img.convert("L")
    img = ImageOps.autocontrast(img, cutoff=1)
    img = img.filter(ImageFilter.MedianFilter(size=3))
    return img

def extract_text_from_pdf(pdf_path, ocr_if_needed=True, min_text_len_for_layer=50, dpi=300):
    all_text = []
    with pdfplumber.open(pdf_path) as pdf:
        for page_index, page in enumerate(pdf.pages):
            try:
                text = page.extract_text()
            except Exception as e:
                print(f"page.extract_text() error on page {page_index+1}: {e}")
                text = None

            if (not text or len(text.strip()) < min_text_len_for_layer) and ocr_if_needed:
                print(f"Page {page_index+1}: Running OCR at {dpi} DPI...")
                try:
                    page_image = page.to_image(resolution=dpi).original
                except Exception as e:
                    print(f"page.to_image error fallback for page {page_index+1}: {e}")
                    page_image = page.to_image().original
                page_image = preprocess_pil_image_for_ocr(page_image)
                try:
                    text = pytesseract.image_to_string(page_image, config=TESSERACT_COMMON_CONFIG)
                except Exception as e:
                    print(f"Tesseract OCR failed on page {page_index+1}: {e}")
                    text = ""
            if not text:
                text = ""
            all_text.append(f"\\n--- PAGE {page_index+1} ---\\n{text}")
    return "\\n".join(all_text)

def extract_text_from_docx(path):
    try:
        doc = docx.Document(path)
        paragraphs = [p.text for p in doc.paragraphs if p.text and p.text.strip()]
        return "\\n".join(paragraphs)
    except Exception as e:
        print("DOCX read error:", e)
        return ""

def extract_text_from_image(path):
    try:
        img = Image.open(path)
        img = preprocess_pil_image_for_ocr(img)
        return pytesseract.image_to_string(img, config=TESSERACT_COMMON_CONFIG)
    except Exception as e:
        print("Image OCR error:", e)
        return ""


In [ ]:
# Upload files (Colab interactive)
from google.colab import files
uploaded = files.upload()
uploaded_filenames = list(uploaded.keys())
print("Uploaded:", uploaded_filenames)

In [14]:
# Chunking, process files to JSON and prepare records
def chunk_text(text, chunk_size=300, overlap=50):
    tokens = text.split()
    chunks = []
    i = 0
    while i < len(tokens):
        chunk_tokens = tokens[i:i+chunk_size]
        chunks.append(" ".join(chunk_tokens))
        i += chunk_size - overlap
    return chunks

def process_and_export(files_list):
    records = []
    for fname in files_list:
        path = Path(fname)
        ext = path.suffix.lower()
        if ext == ".pdf":
            raw = extract_text_from_pdf(str(path))
        elif ext in [".docx", ".doc"]:
            raw = extract_text_from_docx(str(path))
        elif ext in [".png", ".jpg", ".jpeg", ".tif", ".tiff", ".bmp"]:
            raw = extract_text_from_image(str(path))
        else:
            print("Skipping unsupported:", fname)
            continue

        if not raw or len(raw.strip()) == 0:
            print("No text extracted for:", fname)
            continue

        metadata = {"source_filename": fname, "id": str(uuid.uuid4()), "n_chars": len(raw)}
        chunks = chunk_text(raw, chunk_size=300, overlap=50)
        doc_json = {"metadata": metadata, "full_text": raw, "chunks": []}
        for idx, chunk in enumerate(chunks):
            chunk_id = f"{metadata['id']}_chunk_{idx}"
            doc_json["chunks"].append({"chunk_id": chunk_id, "text": chunk, "chunk_index": idx})
            records.append({"id": chunk_id, "text": chunk, "metadata": {**metadata, "chunk_index": idx}})
        outpath = Path(JSON_OUTPUT_DIR) / (path.stem + ".json")
        with open(outpath, "w", encoding="utf-8") as f:
            json.dump(doc_json, f, ensure_ascii=False, indent=2)
        print("Wrote JSON:", outpath)
    return records

# Run processing on uploaded files
records = process_and_export(uploaded_filenames)
print("Total chunks prepared:", len(records))

Wrote JSON: /content/doc_jsons/Uniform Residential Loan Application details.json
Total chunks prepared: 2


In [15]:
# Load embedding model and create embeddings
embed_model = SentenceTransformer(EMBEDDING_MODEL_NAME)
texts = [r["text"] for r in records]
ids = [r["id"] for r in records]
metadatas = [r["metadata"] for r in records]

BATCH = 64
embeddings = []
for i in tqdm(range(0, len(texts), BATCH), desc="Embedding"):
    batch_texts = texts[i:i+BATCH]
    embs = embed_model.encode(batch_texts, show_progress_bar=False, convert_to_numpy=True)
    embeddings.extend(embs)
print("Embeddings vectors:", len(embeddings))

Embedding: 100%|██████████| 1/1 [00:03<00:00,  3.50s/it]

Embeddings vectors: 2


In [16]:
# Create Chroma DB and insert vectors
client = chromadb.PersistentClient(path=DB_DIR, settings=Settings())
collection_name = "customer_docs"
try:
    collection = client.get_or_create_collection(collection_name)
except Exception:
    collection = client.create_collection(name=collection_name)

# prepare embeddings to lists of floats (Chroma expects python floats)
vecs = [e.tolist() if hasattr(e, "tolist") else list(map(float, e)) for e in embeddings]

collection.add(ids=ids, metadatas=metadatas, documents=texts, embeddings=vecs)
print("Chroma persisted at", DB_DIR)

Chroma persisted at /content/chroma_db


In [17]:
# Retrieval helper
def search_chroma(query_text, top_k=3):
    q_emb = embed_model.encode([query_text], convert_to_numpy=True)[0].astype(float).tolist()
    results = collection.query(query_embeddings=[q_emb], n_results=top_k, include=["documents","metadatas","distances"])
    hits = []
    for i in range(len(results["ids"][0])):
        hits.append({
            "id": results["ids"][0][i],
            "document": results["documents"][0][i],
            "metadata": results["metadatas"][0][i],
            "distance": results["distances"][0][i]
        })
    return hits

In [18]:
# Load generation model (FLAN-T5) for RAG fusion
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Generation device:", device)

gen_tokenizer = AutoTokenizer.from_pretrained(GEN_MODEL_NAME)
gen_model = AutoModelForSeq2SeqLM.from_pretrained(GEN_MODEL_NAME).to(device)

def tokenize_len(text):
    return len(gen_tokenizer.encode(text, truncation=False))
def truncate_context_by_tokens(chunks_texts, max_input_tokens, sep="\n\n---\n\n"):
    out = []
    tokens = 0
    for t in chunks_texts:
        t_tokens = len(gen_tokenizer.encode(t + sep, truncation=False))
        if tokens + t_tokens > max_input_tokens:
            break
        out.append(t)
        tokens += t_tokens
    return sep.join(out)

Generation device: cpu


In [19]:
# Prompt template and RAG answer function
PROMPT_SYSTEM = """You are a helpful assistant. Answer the user question using ONLY the CONTEXT provided.
If the context does not contain enough information to answer, respond: INSUFFICIENT_CONTEXT.
Be concise (max ~120 words). Provide the answer and then list the sources (filename and chunk index)."""

PROMPT_USER_TEMPLATE = """
CONTEXT:
{context}

QUESTION:
{question}

INSTRUCTIONS:
1) Give a short direct answer (<120 words).
2) After the answer print a "SOURCES:" section listing each source as - filename (chunk_index).
3) If you can't answer from the context, respond exactly: INSUFFICIENT_CONTEXT
"""

def build_prompt(context, question):
    return PROMPT_SYSTEM + "\n\n" + PROMPT_USER_TEMPLATE.format(context=context, question=question)

def rag_answer(question, top_k=3, max_context_tokens=1500, max_answer_tokens=180):
    # 1. Retrieve
    hits = search_chroma(question, top_k=top_k)
    if not hits:
        return {"question": question, "answer": "INSUFFICIENT_CONTEXT", "provenance": [], "used_context": ""}

    # 2. Build entries with provenance
    entries = []
    provenance = []
    for h in hits:
        src = h['metadata'].get('source_filename', 'unknown')
        idx = h['metadata'].get('chunk_index', -1)
        entry_text = f"[{src} | chunk {idx}]\n{h['document']}"
        entries.append(entry_text)
        provenance.append({"source": src, "chunk_index": idx, "distance": h['distance'], "id": h['id']})

    # 3. Truncate context to token budget
    context = truncate_context_by_tokens(entries, max_context_tokens, sep="\n\n---\n\n")
    prompt = build_prompt(context, question)

    # 4. Tokenize & generate
    inputs = gen_tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048).to(device)
    outputs = gen_model.generate(
        **inputs,
        max_new_tokens=max_answer_tokens,
        num_beams=4,
        do_sample=False,
        early_stopping=True
    )
    answer = gen_tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    if not answer:
        answer = "INSUFFICIENT_CONTEXT"

    return {"question": question, "answer": answer, "provenance": provenance, "used_context": context}

In [20]:
# Optional: QA span extraction for exact fields
qa_pipeline = pipeline("question-answering", model=QA_MODEL_NAME, tokenizer=QA_MODEL_NAME, device=0 if torch.cuda.is_available() else -1)

def extract_exact_span(question, top_hit):
    context = top_hit['document']
    res = qa_pipeline(question=question, context=context, top_k=1)
    return res

Device set to use cpu


In [25]:
# Examples / Usage

# Retrieval-only example:
q = "list of all the customers with monthly income greater than 5000 dollars?"
hits = search_chroma(q, top_k=3)
for i,h in enumerate(hits,1):
    print(f"HIT {i}: {h['metadata']['source_filename']} chunk {h['metadata']['chunk_index']}")
    print(h['document'][:500].replace("\n"," "))
    print("distance:", h['distance'])
    print("-----\n")

# RAG: generate concise, grounded answer
out = rag_answer(q, top_k=3)
print("ANSWER:\n", out["answer"])
print("\nPROVENANCE:\n", out["provenance"])

# Optional: exact span from top hit
if hits:
    span = extract_exact_span(q, hits[0])
    print("\nExact span extracted by QA model:", span)

HIT 1: Customer information.docx chunk 0
Page 1: Borrower Information\nSection 1a. Personal Information \nName: James A. Smith \nSocial Security Number: 000-XX-1234 \nDate of Birth: 06/15/1985 \nCitizenship: Check Box: U.S. Citizen \nMarital Status: Check Box: Married \nDependents: 1 | Ages: 5 \nCurrent Address: 4509 Maple Drive \nCity/State/Zip: Austin, TX 78701 \nHousing: Check Rent ($1,800/month) \nSection 1b. Employment/Income \nEmployer Name: Tech Solutions Inc. \nGross Monthly Income (Base): $8,500 \nBonus: $500 \nTotal Income: $
distance: 1.2032997608184814
-----

HIT 2: Uniform Residential Loan Application details.docx chunk 0
Uniform Residential Loan Application Summary 3\nBorrower Name: JACK MERIDITH SPECTOR 4444Lender Loan No/Universal Loan Identifier: US BANK 5Agency Case No.: US-100 6Date of Application Signature: 11/10/2025 7\nSection 1: Borrower Information 8\n1a. Personal Information\nSocial Security Number: 15 52 556 (or Individual Taxpayer Identification Number) 9999\

In [ ]:
# Save DB and JSONs for download (optional)
!zip -r -q /content/chroma_db.zip /content/chroma_db
!zip -r -q /content/doc_jsons.zip /content/doc_jsons
print("Zipped at /content/chroma_db.zip and /content/doc_jsons.zip")

Zipped at /content/chroma_db.zip and /content/doc_jsons.zip
